In [1]:
import torch
import torch.nn as nn
import os
import numpy as np
import random
import importlib

# custom imports
import dataset

In [2]:
def load_lookup_table(model_name):
    assert model_name in ["FNN", "RNN", "LSTM"], "model name must be one of FNN, RNN, LSTM"
    
    if not os.path.exists("checkpoint"):
        os.mkdir("checkpoint")

    pth_list = os.listdir("checkpoint")
    latest_pth = None
    for pth in pth_list:
        if pth.endswith(".pth") and pth.startswith(model_name):
            if latest_pth is None:
                latest_pth = pth
            else:
                id = int(pth.split("-")[-1].split(".")[0])
                latest_id = int(latest_pth.split("-")[-1].split(".")[0])
                if id > latest_id:
                    latest_pth = pth

    if latest_pth is not None:
        print("load model from checkpoint/" + latest_pth)
        lookup_table = torch.load("checkpoint/" + latest_pth)['embedding.weight'].cpu().numpy()
        print("lookup table shape:", lookup_table.shape)
    
    return lookup_table


In [3]:
def load_top_words():
    data = dataset.Dataset()
    data.load("data_rnn.pkl")
    return data.top_words

In [4]:
def get_word(top_words, word_idx):
    name_list = [key for key,value in top_words.items() if value==word_idx]
    if len(name_list)>0:
        return name_list[0]
    else:
        return "<UNK>"

def get_similar_wordidx(lookup_table, word_idx):
    word_vec = lookup_table[word_idx]
    # print(word_vec.shape)
    # print(np.dot(lookup_table, word_vec).shape)
    # print(np.linalg.norm(lookup_table, axis=1).shape)
    
    normed_lookup_table = lookup_table/np.linalg.norm(lookup_table, axis=1).reshape((-1,1))
    normed_word_vec = word_vec/np.linalg.norm(word_vec)
    similarity = np.dot(normed_lookup_table, normed_word_vec)

    order = np.argsort(-similarity)
    return order, similarity

def top_10_similar(top_words, lookup_table, word_idx):
    a, similarity = get_similar_wordidx(lookup_table, word_idx)
    for i in a[:10]:
        print("|", get_word(top_words, i), similarity[i])
    # print("top 10 opposite words:")
    # for i in a[:-10:-1]:
    #     print("|", get_word(i), similarity[i])
    # print("least 10 similar words:")
    # b = np.argsort(np.abs(similarity))
    # for i in b[:10]:
    #     print("|", get_word(i), similarity[i])

In [5]:
top_words = load_top_words()

word_list = []
for i in range(20):
    word = random.choice(list(top_words.keys()))
    word_list.append(word)
print("choosing word list:", word_list)

lookup_table_dict = {
    "FNN": load_lookup_table("FNN"),
    "RNN": load_lookup_table("RNN"),
    "LSTM": load_lookup_table("LSTM")
}

for word in word_list:
    print("--------------------")
    print(word)
    for model_name in ["FNN", "RNN", "LSTM"]:
        lookup_table = lookup_table_dict[model_name]
        print("top 10 similar words in "+model_name+":")
        top_10_similar(top_words, lookup_table, top_words[word])

choosing word list: ['[中央/n', '腐败/an', '开放/v', '没有/v', '标准/n', '？/w', '基础/n', '思想/n', '[北京/ns', '虽然/c', '而/c', '泽民/nr', '提高/v', '特别/d', '报道/v', '形式/n', '创造/v', '外国/n', '股份制/n', '其/r']
load model from checkpoint/FNN-60.pth
lookup table shape: (1024, 16)
load model from checkpoint/RNN-60.pth
lookup table shape: (1024, 16)
load model from checkpoint/LSTM-60.pth
lookup table shape: (1024, 16)
--------------------
[中央/n
top 10 similar words in FNN:
| [中央/n 1.0
| [国家/n 0.96248627
| [中国/ns 0.9283081
| [全国/n 0.92518127
| 英国/ns 0.8885466
| [上海/ns 0.86471623
| 俄罗斯/ns 0.85656345
| 自治区/n 0.8472166
| 韩国/ns 0.8465545
| 泰国/ns 0.84284866
top 10 similar words in RNN:
| [中央/n 1.0
| [国家/n 0.9614538
| [中国/ns 0.9366956
| [北京/ns 0.8813087
| [上海/ns 0.84907675
| 自治区/n 0.8306463
| 泰国/ns 0.809421
| 韩国/ns 0.7967012
| [中共中央/nt 0.79579103
| [美国/ns 0.79564255
top 10 similar words in LSTM:
| [中央/n 0.9999999
| [国家/n 0.9378362
| [全国/n 0.8085585
| 国务院/nt 0.76516944
| [北京/ns 0.76076376
| [上海/ns 0.7285284
| [中国/ns 0.7259

In [11]:
for model_name in ["FNN", "RNN", "LSTM"]:
    lookup_table = lookup_table_dict[model_name]
    print("--------------------")
    print("top similar word pair in "+model_name+":")
    normed_lookup_table = lookup_table/np.linalg.norm(lookup_table, axis=1).reshape((-1,1))
    similarity = np.dot(normed_lookup_table, normed_lookup_table.T)
    a=(-similarity.flatten()).argsort()
    r,c = divmod(a, similarity.shape[1])
    for i in range(0,98,2):
        print(get_word(top_words, r[1024+i]), 
              get_word(top_words, c[1024+i]), 
              similarity[r[1024+i]][c[1024+i]])

--------------------
top similar word pair in FNN:
张/nr 陈/nr 0.99425304
陈/nr 王/nr 0.9939114
王/nr 张/nr 0.99198544
陈/nr 刘/nr 0.9916302
赵/nr 陈/nr 0.9874748
王/nr 刘/nr 0.9873147
张/nr 赵/nr 0.9858576
台/q 座/q 0.98276216
杨/nr 陈/nr 0.9813833
张/nr 刘/nr 0.9808551
吴/nr 刘/nr 0.9785289
张/nr 杨/nr 0.9773868
王/nr 赵/nr 0.9767954
刘/nr 赵/nr 0.97665775
赵/nr 杨/nr 0.9741169
王/nr 杨/nr 0.9721345
王/nr 马/nr 0.9708788
４/m ８/m 0.9699005
陈/nr 吴/nr 0.96990025
刘/nr 杨/nr 0.9695685
马/nr 陈/nr 0.9694613
吴/nr 胡/nr 0.9677454
四/m 五/m 0.9656019
赵/nr 马/nr 0.9629137
孙/nr 赵/nr 0.96266186
[中央/n [国家/n 0.9624863
马/nr 刘/nr 0.96241826
马/nr 张/nr 0.96233374
吴/nr 王/nr 0.95954776
吴/nr 杨/nr 0.9590762
黄/nr 何/nr 0.9586615
黄/nr 孙/nr 0.9579629
黄/nr 吴/nr 0.9569227
完善/v 改善/v 0.9544951
位/q 场/q 0.95434386
张/nr 吴/nr 0.95425975
座/q 场/q 0.9540532
刘/nr 黄/nr 0.95376205
徐/nr 刘/nr 0.95307344
陈/nr 黄/nr 0.9521703
孙/nr 陈/nr 0.95209694
全市/n 全省/n 0.9517855
胡/nr 刘/nr 0.95138985
孙/nr 刘/nr 0.95027417
徐/nr 林/nr 0.9501083
朱/nr 李/nr 0.9493868
杨/nr 马/nr 0.9491738
罗

In [15]:
# generate the top 10 similar words for each word in the vocabulary
for model_name in ["FNN", "RNN", "LSTM"]:
    lookup_table = lookup_table_dict[model_name]
    print("--------------------")
    print("top 10 similar word in "+model_name+" for every word:")
    words = list(top_words.keys())
    for word in words:
        print((word+":").ljust(6), end="\t")
        order, similarity = get_similar_wordidx(lookup_table, top_words[word])
        for i in order[1:10]:
            print(get_word(top_words, i), end=" ")
        print()
    

--------------------
top 10 similar word in FNN for every word:
，/w:  	；/w 。/w ……/w 并/c ！/w ——/w 是否/v 虽然/c 才/d 
的/u:  	主席/n 左右/m 了/y 居民/n 实行/v 发表/v 使用/v 成立/v 现象/n 
。/w:  	？/w ；/w ：/w ！/w ……/w ——/w ，/w 了/y 自/p 
、/w:  	和/c 与/c 及/c 等/u 联合/v 为/v 以及/c 为主/v 大学/n]nt 
在/p:  	从/p 根据/p 按照/p 由于/c 作为/p 与/p 通过/p 来自/v 对于/p 
和/c:  	与/c 、/w 及/c 等/u 进程/n 联合/v 以及/c 为主/v 统一/vn 
了/u:  	出/v 着/u 过/u 得/u 发/v 到/v 起/v 处理/v 上/v 
是/v:  	既/c 不仅/c 只/d 无/v 都/d 创造/v 给/p 或者/c 却/d 
“/w:  	『/w 英国/ns 才/c 灾区/n ）/w [上海/ns 地区/n 上海/ns 而且/c 
”/w:  	』/w ’/w 老/a 行动/vn 现象/n 职工/n 速度/n 女/b 目标/n 
一/m:  	那/r 无/v 那些/r 张/q 买/v 这种/r 首/m 变/v 请/v 
有/v:  	在/v 像/v 没有/v 成为/v 无/v 买/v 可以/v 面对/v 于/p 
不/d:  	很/d 要/v 太/d 没有/v 十分/m 更/d 应该/v 多少/r 走/v 
（/w:  	图片/n 』/w ”/w 新华社/nt 图/n 十五大/j 附/v 钱/nr 以及/c 
）/w:  	美元/q 讲话/n 协会/n]nt 演出/v 总统/n 有限公司/n]nt 同志/n 英国/ns 委员/n 
对/p:  	同/p 与/p 为/p 按照/p 把/p 关于/p 为了/p 由于/p 反对/v 
：/w:  	。/w ……/w 时/Ng ？/w 我/r ！/w 这种/r 着/u 这样/r 
中/f:  	上/f 里/f 间/f 下/f 方面/n 之间/f 时/Ng 内/f 年代/n 
这/r:  	尤其/d 特别/d 人们/n 如果/c 每/r 它/r 党/n 过去